In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", 0)

# https://www.tizi365.com/topic/10092.html
model_local_path ='/Users/jiayixian/projects/llm/llm/models/bge-large-en-v1.5'
model = SentenceTransformer(model_local_path)
label = ""
data_path = f"/Users/jiayixian/projects/llm/data/{label}.xlsx" 
# sentences clustering
df = pd.read_excel(data_path)
lst_text = df['utterance'].tolist()

/usr/local/Caskroom/miniconda/base/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from utils import *
# https://www.tizi365.com/topic/10092.html
model_local_path ='/Users/jiayixian/projects/llm/llm/models/bge-large-en-v1.5'
#model = SentenceTransformer(model_local_path)
data_path = "" 
# sentences clustering
df = pd.read_excel(data_path)
label_col = 'my_label'
col = 'utterance'

guides_lst = txt2lst(f"/Users/jiayixian/projects/llm/llm/{label}.txt")

top_k_labels = df[label_col].value_counts().head(8).index.tolist() # Series
print(top_k_labels)

label_lst = top_k_labels
# ['statement_balance', '.total_balance', 'balance_x', 'balance_issue_incorrect', 
# 'balance_issue_incorrect0', '.payment_balance_confirm']

#mat_dict_combined, text_dict=get_txt_lst(model, label_lst, df)
#label2st= get_label2st(label_lst, df, label_col='my_label', col='utterance')
#mat_crx_dict = get_cross_matrix_dict(label_lst, label2st, mat_dict_combined['combined'])

label_lst = []
# label_lst = ['balance_issue_incorrect', 'balance_issue_incorrect0']
samples = generate_label2samples(model, df, label_lst, topk=1, Is_combined=True, path="", label_col='my_label', col='utterance') # text_dict, label2st, mat_dict_combined, mat_crx_dict, 

import json
label_joined_name = "_".join(label_lst)

path = "/Users/jiayixian/projects/llm/llm/samples_dict.json"

with open(path, "r") as f:
    label2samples_dict=json.load(f)
f.close()

if label_joined_name not in label2samples_dict:
    label2samples_dict[label_joined_name] = []
label2samples_dict[label_joined_name].append(samples)

with open(path, "w") as f:
    json.dump(label2samples_dict,f)
f.close()

In [ ]:
guides_lst = txt2lst("/Users/jiayixian/projects/llm/data/due_data.txt")
label_col = 'my_label'
col = 'utterance'
top_k_labels = df[label_col].value_counts().head(8).index.tolist() # Series
print(top_k_labels)

#txt_samples = [text_dict['.due_date'][idx] for idx in min_row]
#m = get_similarity(guides_lst, txt_samples)

#### utils

In [97]:
from collections import Counter
from collections import defaultdict

def txt2lst(path):
    with open(path, "r") as f:
        lines = f.readlines()
        lines = [line.strip().split('. ', 1)[1] if line.strip().split('. ', 1)[0].isdigit() else line.strip() for line in lines]
        return lines

def get_similarity(lst1, lst2):
    embeddings_1 = model.encode(lst1, normalize_embeddings=True)
    embeddings_2 = model.encode(lst2, normalize_embeddings=True)
    similarity = embeddings_1 @ embeddings_2.T
    return similarity

def get_max_min_col_row(data,Is_same_label=False, Is_print=True):

    N, M = data.shape[0], data.shape[1]
    if Is_same_label:
        data = data-np.eye(N,M)

    aver_row = np.sum(data, axis=0)/data.shape[0]
    aver_col = np.sum(data, axis=1)/data.shape[1]
    aver = np.sum(data)/(data.shape[0]*data.shape[1])

    
    # Calculate max, min, and average for each row
    max_per_row = np.max(data, axis=1)
    min_per_row = np.min(data, axis=1)
    avg_per_row = np.mean(data, axis=1)

    # Calculate max, min, and average for each column
    max_per_col = np.max(data, axis=0)
    min_per_col = np.min(data, axis=0)
    avg_per_col = np.mean(data, axis=0)

    # Find the index of max and min for each row
    max_idx_per_row = np.argmax(data, axis=1)
    min_idx_per_row = np.argmin(data, axis=1)

    # Find the index of max and min for each column
    max_idx_per_col = np.argmax(data, axis=0)
    min_idx_per_col = np.argmin(data, axis=0)

    # Find the second largest value in each column
    second_largest_in_columns = np.partition(data, -2, axis=0)[-2]

    # Find the second largest value in each row
    second_largest_in_rows = np.partition(data, -2, axis=1)[:, -2]
    
    if Is_same_label:
        avg_per_col = np.sum(data, axis=0)/(N-1)
        avg_per_row = np.sum(data, axis=1)/(M-1)
        aver = np.sum(aver_row)/(N-1)
        min_per_row = np.min(data + np.eye(N,M), axis=1)
        min_per_col = np.min(data + np.eye(N,M), axis=0)
        min_idx_per_row = np.argmin(data + np.eye(N,M), axis=1)
        min_idx_per_col = np.argmin(data + np.eye(N,M), axis=0)
        data = data+np.eye(N,M)
    # Print the results
    if Is_print:
        print(f"aver row: {aver_row}, aver col: {aver_col}, aver: {aver}")
        print("Max per row:", max_per_row)
        print("Min per row:", min_per_row)
        print("Average per row:", avg_per_row)

        print("Max per column:", max_per_col)
        print("Min per column:", min_per_col)
        print("Average per column:", avg_per_col)

        # Print the results
        print("Index of Max per row:", max_idx_per_row)
        print("Index of Min per row:", min_idx_per_row)

        print("Index of Max per column:", max_idx_per_col)
        print("Index of Min per column:", min_idx_per_col)

        print("Second largest value in each column:", second_largest_in_columns)
        print("Second largest value in each row:", second_largest_in_rows)

    d = {}
    d['avg_r'] = avg_per_row
    d['avg_c'] = avg_per_col
    d['min_r'] = min_idx_per_row
    d['min_c'] = min_idx_per_col
    d['max_r'] = max_idx_per_row
    d['max_c'] = max_idx_per_col
    d['avg'] = aver
    d['min'] = np.min(data)
    d['max'] = np.max(data-np.eye(N,M)) if Is_same_label else np.max(data)
    return d

def get_dict_val2idx(A):
    val2idx = defaultdict(list)
    for idx, val in enumerate(A):
        val2idx[int(val)].append(idx)
    return val2idx

def get_topk_within_idx(A, indices, topk):
    selected_values = A[indices]

    # Find the top 5 values and their corresponding indices
    top_indices = selected_values.argsort()[-topk:][::-1]  # Indices of the top k values
    top_values = selected_values[top_indices]

    return top_values, [indices[i] for i in top_indices] # top_indices 是ndarray 不能直接 indices[top_indices]

def get_topk_txt_within_idx(d_minmax, d_idx, topk=3, key='avg_r'):
    A = d_minmax[key]
    d_res = {}
    for key, idxes in d_idx.items():
        top_val, top_idxes = get_topk_within_idx(A, idxes, topk)
        d_res[key] = top_idxes
    return d_res

def visual_res(d_res, text_dict, label1):
    for key, idxes in d_res.items():
        print(key, text_dict[label1][key])
        print([[idx, text_dict[label1][idx]] for idx in idxes])

def txt_collect(d_res, extra_list, d_txt4def, label):
    idx_set = set(extra_list)
    for outliner, txt_lst in d_res.items():
        idx_set = idx_set | set(txt_lst)
        idx_set.add(outliner)
    d_txt4def[label] = sorted(list(idx_set))
    return d_txt4def
def get_txt_lst(label_lst, df, label_col='my_label', col='utterance'):
    lst = []
    text_dict = {}
    for label in label_lst:
        text_dict[label]=df[df[label_col]==label][col].tolist()
        lst += df[df[label_col]==label][col].tolist()
        print(label, len(text_dict[label]))
    mat_dict = {}
    mat_dict['combined'] = get_similarity(lst,lst)
    
    return mat_dict, text_dict

def get_txt_lst_per_label(label_lst, df, top_k_labels, label_col='my_label', col='utterance'):
    mat_dict = {}
    for label in top_k_labels:
        mat_dict[label] = get_similarity(df[df[label_col]==label][col].tolist(),df[df[label_col]==label][col].tolist())

    text_dict = {}
    for label in df['my_label'].unique():
        text_dict[label]=df[df[label_col]==label][col].tolist()
    
    return mat_dict, text_dict


def get_label2st(label_lst, df):
    label2st = {}
    s, t = 0, 0
    for label in label_lst:
        t = s+len(df[df[label_col]==label][col].tolist())
        label2st[label]=[s, t]
        print(f"{label}: [{s}, {t}]")
        s = t
    return label2st

def get_label2matrix(matrix, label2st, label1, label2):
    """
    matrinx: numpy ndarray of (N, N)
    """
    st1, st2 = label2st[label1], label2st[label2]
    return matrix[st1[0]:st1[1], st2[0]:st2[1]]

def get_cross_matrix_dict(label_lst, label2st, matrix):
    mat_dict = {}
    for label1 in label_lst:
        if label1 not in mat_dict:
            mat_dict[label1] = {}
            for label2 in label_lst:
                mat_dict[label1][label2] = get_label2matrix(matrix, label2st, label1, label2)
    return mat_dict

#### run

In [173]:
#argsort(d_minmax['avg_r'])
kth_smallest_index = np.argpartition(d_minmax['avg_r'], topk+3)[:topk+3]

In [174]:
kth_smallest_index

array([39,  6, 37, 14])

In [ ]:
label_lst = ['statement_balance', '.total_balance', 'balance_x', 'balance_issue_incorrect', 'balance_issue_incorrect0', '.payment_balance_confirm']
mat_dict_combined, text_dict=get_txt_lst(label_lst, df)
label2st= get_label2st(label_lst, df)
mat_crx_dict = get_cross_matrix_dict(label_lst, label2st, mat_dict_combined['combined'])

In [ ]:
def generate_label2samples(label_lst, topk=1, Is_combined=True, path=""):
    label1 = 'statement_balance'
    label2 = '.total_balance' #'statement_balance'
    aver_rel = {label:{} for label in label_lst}

    if Is_combined:
        pass
    for l1 in label_lst:
        for l2 in label_lst:
            d_minmax = get_max_min_col_row(mat_crx_dict[l1][l2], Is_same_label=(l1==l2), Is_print=False)
            aver_rel[l1][l2] = d_minmax
            print(l1, l2, d_minmax['avg'])
            #print(d_minmax['avg_r'])
            #print(d_minmax['avg_c'])

    #d_minmax = get_max_min_col_row(mat_crx_dict[label1][label2], Is_same_label=(label1==label2), Is_print=True)
    # Get outliners
    cnt = Counter(d_minmax['min_r'])
    # Get value 2 idx dictionary
    v2idx = get_dict_val2idx(A=d_minmax['min_r'])
    # get the most centered ones with longest distance w.r.t outliners 
    # for each index in d_idx[val], get the aver_score, select the ones with higest aver_mean
    d_res = get_topk_txt_within_idx(d_minmax, v2idx, topk, key='avg_r')
    #print(d_minmax['min_r'])
    visual_res(d_res, text_dict, label1)
    sorted_outliners = [x[0] for x in sorted([(key, count) for key, count in cnt.items()], key=lambda x: -x[1])]
    print(sorted_outliners)
    d_txt4def = {}
    kth_smallest_index = np.argpartition(d_minmax['avg_r'], topk+3)[:topk+3]
    d_txt4def = txt_collect(d_res, kth_smallest_index, d_txt4def, label1)
    samples = [text_dict[label1][idx] for idx in d_txt4def[label1]]



In [ ]:
label1 = 'statement_balance'
label2 = '.total_balance' #'statement_balance'
topk = 1
aver_rel = {label:{} for label in label_lst}
for l1 in label_lst:
    for l2 in label_lst:
        d_minmax = get_max_min_col_row(mat_crx_dict[l1][l2], Is_same_label=(l1==l2), Is_print=False)
        aver_rel[l1][l2] = d_minmax
        print(l1, l2, d_minmax['avg'])
        #print(d_minmax['avg_r'])
        #print(d_minmax['avg_c'])

#d_minmax = get_max_min_col_row(mat_crx_dict[label1][label2], Is_same_label=(label1==label2), Is_print=True)
# Get outliners
cnt = Counter(d_minmax['min_r'])
# Get value 2 idx dictionary
v2idx = get_dict_val2idx(A=d_minmax['min_r'])
# get the most centered ones with longest distance w.r.t outliners 
# for each index in d_idx[val], get the aver_score, select the ones with higest aver_mean
d_res = get_topk_txt_within_idx(d_minmax, v2idx, topk, key='avg_r')
#print(d_minmax['min_r'])
visual_res(d_res, text_dict, label1)
sorted_outliners = [x[0] for x in sorted([(key, count) for key, count in cnt.items()], key=lambda x: -x[1])]
print(sorted_outliners)
d_txt4def = {}
kth_smallest_index = np.argpartition(d_minmax['avg_r'], topk+3)[:topk+3]
d_txt4def = txt_collect(d_res, kth_smallest_index, d_txt4def, label1)
samples = [text_dict[label1][idx] for idx in d_txt4def[label1]]



In [80]:
mat_dict_combined['combined'].shape

(101, 101)

statement_balance 17
.total_balance 14
balance_x 9
balance_issue_incorrect 23
balance_issue_incorrect0 24
.payment_balance_confirm 24

In [ ]:
# get distant samples for extracting definitions

label = '.due_date'
label = 'combined'
topk = 1
d_minmax = get_max_min_col_row(mat_dict[label], Is_print=True)
# Get outliners
cnt = Counter(d_minmax['min_r'])
# Get value 2 idx dictionary
v2idx = get_dict_val2idx(A=d_minmax['min_r'])
# get the most centered ones with longest distance w.r.t outliners 
# for each index in d_idx[val], get the aver_score, select the ones with higest aver_mean
d_res = get_topk_txt_within_idx(d_minmax, v2idx, topk, key='avg_r')
#print(d_minmax['min_r'])
visual_res(d_res)
sorted_outliners = [x[0] for x in sorted([(key, count) for key, count in cnt.items()], key=lambda x: -x[1])]
print(sorted_outliners)
d_txt4def = {}
kth_smallest_index = np.argpartition(d_minmax['avg_r'], topk+3)[:topk+3]
d_txt4def = txt_collect(d_res, kth_smallest_index, d_txt4def, label)
samples = [text_dict[label][idx] for idx in d_txt4def[label]]
samples


In [164]:
m = get_similarity(samples, samples)
d_samples = get_max_min_col_row(m)

Max per row: [0.9999996  1.0000001  0.99999994 1.0000004  1.0000002  0.99999976
 1.         1.0000007  0.99999976 1.0000001  1.0000004  1.0000004
 1.0000002  1.0000004  0.9999997 ]
Min per row: [0.6455203  0.4944928  0.53733385 0.64476454 0.4944928  0.5288431
 0.6262151  0.5527522  0.60912544 0.5527522  0.668323   0.578886
 0.5428655  0.62845504 0.5288431 ]
Average per row: [0.7800027  0.6410429  0.6724819  0.7794098  0.6563986  0.724314
 0.7242067  0.6689293  0.73428124 0.7136544  0.7904648  0.7355234
 0.6711038  0.72936314 0.6485892 ]
Max per column: [0.9999996  1.0000001  0.99999994 1.0000004  1.0000002  0.99999976
 1.         1.0000007  0.99999976 1.0000001  1.0000004  1.0000004
 1.0000002  1.0000004  0.9999997 ]
Min per column: [0.6455203  0.4944928  0.53733385 0.64476454 0.4944928  0.5288431
 0.6262151  0.5527522  0.60912544 0.5527522  0.668323   0.578886
 0.5428655  0.62845504 0.5288431 ]
Average per column: [0.7800027  0.6410429  0.6724819  0.7794098  0.6563986  0.724314
 0.724